In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/content/Copy of Database Anonymized - DC Motor Q&A - Q1.csv")
df

,S. No.,What is the working principle of a DC motor? List a few important components of a DC motor. (Answers copied verbatim from the internet or other students will be awarded 0 marks.)
0,1,The principle of is that when an current carry...
1,2,The working principle is that when a current c...
2,3,"Components of DC motor are: rotor, stator, bru..."
3,4,The basic principle of dc motor is that it exp...
4,5,When a current carrying conductor is placed in...
...,...,...
219,220,The working principle of DC motor is that a cu...
220,221,The principle behind an electric DC motor is t...
221,222,The basic working principle is that when a cur...
222,223,The basic working principle of the dc motor is...


In [3]:
second_column = df.iloc[0:, 1]
# second_column

# 1) Length of answer

In [4]:
def grade_answers(answer):
        if len(answer.split()) < 5:
            grades = 0
            flag = "Answer is too short"
        else:
            flag = "Continue for further evaluation"
        return flag


In [5]:
CheckLength = []
for val in second_column:
    if isinstance(val, float):
        # Handle float values
        CheckLength.append(grade_answers(str(val)))
    else:
        CheckLength.append(grade_answers(val))
# print(CheckLength)


In [6]:

# Create a new column with the CheckLength values
df['CheckLength'] = pd.Series(CheckLength[:len(df)], index=df.index)


In [7]:
# Apply conditional formatting
def highlight_short_answer(row):
    if row['CheckLength'] == 'Answer is too short':
        return ['background-color: red'] * len(row)
    else:
        return [''] * len(row)

styled_df = df.style.apply(highlight_short_answer, axis=1)

# Display the styled DataFrame
# styled_df

#2) Checking plaig from peers

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(answer1, answer2):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the answers into TF-IDF feature vectors
    tfidf_matrix = vectorizer.fit_transform([answer1, answer2])

    # Calculate the cosine similarity between the TF-IDF vectors
    similarity_matrix = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    # Retrieve the cosine similarity score
    cosine_similarity_score = similarity_matrix[0][0]

    return cosine_similarity_score


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Calculate TF and IDF ranges
tf_range = []
idf_range = []

for i in range(len(second_column)):
    for j in range(i + 1, len(second_column)):
        sentence1 = second_column[i]
        sentence2 = second_column[j]

        # Check for np.nan values
        if pd.isna(sentence1) or pd.isna(sentence2):
            continue

        # Fit and transform the sentences into TF-IDF feature vectors
        tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])

        # Retrieve the TF and IDF values
        tf_values = tfidf_matrix.toarray()
        idf_values = vectorizer.idf_

        # Calculate the TF and IDF ranges for each term
        term_tf_range = [(min(tf_values[:, k]), max(tf_values[:, k])) for k in range(tf_values.shape[1])]
        term_idf_range = (min(idf_values), max(idf_values))

        # Append the term ranges to the overall ranges
        tf_range.extend(term_tf_range)
        idf_range.append(term_idf_range)

# Calculate the overall TF and IDF ranges
overall_tf_range = (min([tf[0] for tf in tf_range]), max([tf[1] for tf in tf_range]))
overall_idf_range = (min([idf[0] for idf in idf_range]), max([idf[1] for idf in idf_range]))

# Print the TF and IDF ranges
print("TF Range:", overall_tf_range)
print("IDF Range:", overall_idf_range)


TF Range: (0.0, 0.7681689488893733)
IDF Range: (1.0, 1.4054651081081644)


In [11]:
peers = []
for i in range(len(second_column)):
    mx_similarity_score = 0
    for j in range(i + 1, len(second_column)):
        sentence1 = second_column[i]
        sentence2 = second_column[j]

        # Check for np.nan values
        if pd.isna(sentence1) or pd.isna(sentence2):
            continue

        similarity_score = calculate_cosine_similarity(sentence1, sentence2)
        mx_similarity_score = max(mx_similarity_score, similarity_score)

    if mx_similarity_score > 0.9:
        grades = 0
        flag = "Answer is copied from a classmate"
    else:
        flag = "Continue for further evaluation"

    peers.append(flag)


In [ ]:
# len(peers)

In [12]:

# Create a new column with the peer values
df['from Peers'] = pd.Series(peers[:len(df)], index=df.index)

In [13]:
# Apply conditional formatting
def highlight_short_answer(row):
    if row['from Peers'] == "Answer is copied from a classmate":
        return ['background-color: red'] * len(row)
    else:
        return [''] * len(row)

styled_df = df.style.apply(highlight_short_answer, axis=1)

# Display the styled DataFrame
# styled_df

#4) Checking keywords

In [15]:

given_keywords = ["Torque" , "current" , "conductor", "armature" , "field" , "commutator", "brush", "electric" , "magnetic" ]


def calculate_keyword_marks(answer, keywords):
    answer = answer.lower()  # Convert the answer to lowercase for case-insensitive comparison
    total_keywords = len(keywords)
    found_keywords = 0

    for keyword in keywords:
        keyword = keyword.lower()  # Convert the keyword to lowercase for case-insensitive comparison
        if keyword in answer:
            found_keywords += 1
        elif keyword + 's' in answer:
            found_keywords += 1
        elif keyword + "es" in answer:
            found_keywords += 1

    prct = (found_keywords / total_keywords) * 100
    marks = 0
    if prct>=90:
      marks = 1.5
    elif prct>=80 and prct<90:
      marks = 1.25
    elif prct>=70 and prct<80:
      marks = 1
    elif prct>=60 and prct<70:
      marks = 0.75
    elif prct>=50 and prct<60:
      marks = 0.5
    elif prct>=10 and prct<50:
      marks = 0.25
    return marks


In [16]:
keywords = []
for i in range(len(second_column)):
  answer = second_column[i]
  if isinstance(answer, float):
    marks = calculate_keyword_marks(str(answer), given_keywords)
  else:
    marks = calculate_keyword_marks(answer, given_keywords)
  keywords.append(marks)



In [ ]:
# keywords

In [ ]:
# len(keywords)

In [17]:
# Create a new column with the CheckLength values
df['Keywords (1.5 marks)'] = pd.Series(keywords[:len(df)], index=df.index)

# 5) Semantic similarity

In [ ]:
!pip install sentence-transformers


In [20]:
from sentence_transformers import SentenceTransformer, util

def calculate_similarity(answer1, answer2):
    # Load the pre-trained model
    model = SentenceTransformer('bert-base-nli-mean-tokens')

    # Encode the answers into their corresponding embeddings
    embeddings = model.encode([answer1, answer2], convert_to_tensor=True)

    # Calculate cosine similarity between the embeddings
    cosine_similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])

    return cosine_similarity.item()

IdealAnswer = "Principle: When a armature rotates, magnetic flux through it changes. When electric current is flowing through it, it experience a force, which can generate a rotating effect. Few important components are: armature, brushes, magnets around the armature, commutator"



In [ ]:
Semantic = []
for i in range(len(second_column)):
  answer = second_column[i]
  if isinstance(answer, float):
    score = calculate_similarity(IdealAnswer, str(answer))
    marks = 0
    if score>=.90:
      marks = 3.5
    elif score>=.80 and score<.90:
      marks = 3
    elif score>=.70 and score<.80:
      marks = 2.5
    elif score>=.60 and score<.70:
      marks = 2.0
    elif score>=.50 and score<.60:
      marks = 1.5
    elif score>=.30 and score<.50:
      marks = 1
    elif score>=.10 and score<.30:
      marks = 0.5
  else:
    score = calculate_similarity(IdealAnswer, answer)
    marks = 0
    if score>=.90:
      marks = 3.5
    elif score>=.80 and score<.90:
      marks = 3
    elif score>=.70 and score<.80:
      marks = 2.5
    elif score>=.60 and score<.70:
      marks = 2.0
    elif score>=.50 and score<.60:
      marks = 1.5
    elif score>=.30 and score<.50:
      marks = 1
    elif score>=.10 and score<.30:
      marks = 0.5
  Semantic.append(marks)

In [ ]:
# Semantic

In [22]:
# Create a new column with the CheckLength values
df['Semantic (3.5 marks)'] = pd.Series(Semantic[:len(df)], index=df.index)

# Total Marks

In [23]:
keyword = df.iloc[0:, 4]
semantic = df.iloc[0:, 5]
# keyword
# semantic

In [24]:
total = []
for i in range(len(keyword)):
  keyMarks = keyword[i]
  semMarks = semantic[i]
  marks = keyMarks + semMarks
  if keyMarks==0 or semMarks==0:
    marks = 0
  total.append(marks)

In [ ]:
# total

In [25]:
# Create a new column with the CheckLength values
df['Total (5 marks)'] = pd.Series(total[:len(df)], index=df.index)

In [27]:
# Write the modified DataFrame back to a CSV file
df.to_csv('your_modified_file.csv', index=False)